# Train a logistic regression for classification using PyTorch

#### Generate the dataset: 1-dimensional feature vector and 10,000 training samples 

In [20]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split

iris = datasets.load_iris()
list(iris.keys())

['data',
 'target',
 'frame',
 'target_names',
 'DESCR',
 'feature_names',
 'filename',
 'data_module']

In [21]:
print(iris.target_names)
print(iris.feature_names)

['setosa' 'versicolor' 'virginica']
['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']


In [22]:
iris["data"]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [23]:
iris["target"]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

#### Convert the dataset into Tensor used by PyTorch

In [24]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

class ConvertDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform
    
    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        feature = self.X[idx]
        label = self.y[idx]
        sample = {'feature': feature, 'label': label}

        if self.transform:
            sample = self.transform(sample)

        return sample

class ToTensor(object):
    def __call__(self, sample):
        feature, label = sample['feature'], sample['label'] 
        label = np.array(label)
        return {'feature': torch.from_numpy(feature).float(),
                'label': torch.from_numpy(label).long()}
    
#Convert training data
train_dataset = ConvertDataset(iris["data"],
                               iris["target"],
                               transform=transforms.Compose([
                                   ToTensor()
                               ]))

#Load the converted training data into train_dataLoader
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=0)

#### Use PyTorch to build a linear regression model

In [25]:
class Logisticegression(nn.Module):
    def __init__(self):
        super(Logisticegression, self).__init__()
        self.fc = nn.Linear(4, 3) #The first "4" specifies that the feature dimension is 4, and the second "3" specifies that this is 3-class classification
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        y = self.fc(x) #The output is a 3-dimentional vector: each presenting a prability for the corresponding class
        y = self.softmax(y)
        
        return y

#### Set up some hyperparameters: use mean squared loss, mini-batch with Adam optimizer, and epochs

In [26]:
epochs = 50
learning_rate = 0.01
lossfunction = nn.CrossEntropyLoss() #Cross entropy loss

model = Logisticegression()
optimizer = optim.Adam(model.parameters(), lr=learning_rate) #Using Adam optimizer

print(model)

Logisticegression(
  (fc): Linear(in_features=4, out_features=3, bias=True)
  (softmax): Softmax(dim=1)
)


#### Train the model

In [27]:
#Define the training function
def train(epoch, model, train_dataloader, optimizer, lossfunction):
    model.train()
    
    train_loss = 0.0
    train_total, train_correct = 0.0, 0.0 
    
    for i, data in enumerate(train_dataloader):
        X, y = data['feature'], data['label'] 

        optimizer.zero_grad()

        predictions = model(X)

        loss = lossfunction(predictions, y)
        loss.backward()
        optimizer.step()
        
        #print statistics
        train_loss += loss.item()
        _, train_predicted = torch.max(predictions.data, 1)
        train_total += y.size(0)
        train_correct += (train_predicted == y).sum().item()

    print("epoch (%d): Train accuracy: %.4f, loss: %.3f" % (epoch, train_correct/train_total, train_loss/train_total))
    
    
#Train the model
for epoch in range(1, epochs + 1):
    train(epoch, model, train_dataloader, optimizer, lossfunction)

epoch (1): Train accuracy: 0.3800, loss: 0.133
epoch (2): Train accuracy: 0.5333, loss: 0.123
epoch (3): Train accuracy: 0.6667, loss: 0.116
epoch (4): Train accuracy: 0.6667, loss: 0.111
epoch (5): Train accuracy: 0.6867, loss: 0.108
epoch (6): Train accuracy: 0.9000, loss: 0.106
epoch (7): Train accuracy: 0.6933, loss: 0.104
epoch (8): Train accuracy: 0.8733, loss: 0.102
epoch (9): Train accuracy: 0.8067, loss: 0.101
epoch (10): Train accuracy: 0.9067, loss: 0.100
epoch (11): Train accuracy: 0.8067, loss: 0.099
epoch (12): Train accuracy: 0.9533, loss: 0.098
epoch (13): Train accuracy: 0.9000, loss: 0.098
epoch (14): Train accuracy: 0.9067, loss: 0.097
epoch (15): Train accuracy: 0.9400, loss: 0.096
epoch (16): Train accuracy: 0.9067, loss: 0.095
epoch (17): Train accuracy: 0.9467, loss: 0.095
epoch (18): Train accuracy: 0.9400, loss: 0.094
epoch (19): Train accuracy: 0.9600, loss: 0.094
epoch (20): Train accuracy: 0.9000, loss: 0.094
epoch (21): Train accuracy: 0.9400, loss: 0.093
e

#### Using the trained model for prediction

In [28]:
model.eval()

predicted_probability= model(torch.Tensor([[5.1, 3.5, 1.4, 0.2]]))
print(predicted_probability)

tensor([[9.8840e-01, 1.1601e-02, 1.6928e-06]], grad_fn=<SoftmaxBackward0>)


In [29]:
_, predicted_label = torch.max(predicted_probability.data, 1)
print(predicted_label)
print(iris.target_names[predicted_label])

tensor([0])
setosa
